En el archivo auto.csv se encuentran los siguientes datos de diferentes automóviles:
    Cilindros
    Cilindrada
    Potencia
    Peso
    Aceleración
    Año del coche
    Origen
    Consumo (mpg)
Las unidades de las características de los automóviles no se encuentran en el sistema internacional. La variable “origen” es un código que identifica al país de origen.

Crea un modelo con él para que se pueda estimar el consumo de un vehículo a partir del resto de las variables.

In [1]:
# Preparamos el entorno
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# importamos las librerias
import pandas as pd  # Python Data Analysis Library

# Leemos el fichero
autos_df = pd.DataFrame.from_csv('auto.csv', sep=',', index_col = None, header = 0)
autos_df_ori = autos_df
print(autos_df.describe())
print(autos_df.head())

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
        cylinders  displacement  horsepower       weight  acceleration  \
count  392.000000    392.000000  392.000000   392.000000    392.000000   
mean     5.471939    194.411990  104.469388  2977.584184     15.541327   
std      1.705783    104.644004   38.491160   849.402560      2.758864   
min      3.000000     68.000000   46.000000  1613.000000      8.000000   
25%      4.000000    105.000000   75.000000  2225.250000     13.775000   
50%      4.000000    151.000000   93.500000  2803.500000     15.500000   
75%      8.000000    275.750000  126.000000  3614.750000     17.025000   
max      8.000000    455.000000  230.000000  5140.000000     24.800000   

       model_year      origin         mpg  
count  392.000000  392.000000  392.000000  
mean    75.979592    1.576531   23.445918  
std      3.683737    0.805518    7.805007  
min     70.000000    1.000000    9.000000  
25%     73.000000

In [2]:
# Obtencion de los Componentes Principales (PCA)
from sklearn.decomposition import PCA

# Lo obtenemos para todas las columnas de datos del fichero que tengan sentido:
# Eliminamos la variable dependiente:
autos_df_y = autos_df.pop('mpg')
print(autos_df.head())
print(autos_df_y.head())
pca = PCA(n_components = len(autos_df.columns))
x_pca = pca.fit_transform(autos_df)
ix = 0
for columna in autos_df.columns:
    print('Varianza explicada por la compontente', autos_df.columns[ix], pca.explained_variance_ratio_[ix])
    ix = ix + 1

   cylinders  displacement  horsepower  weight  acceleration  model_year  \
0          8         307.0       130.0  3504.0          12.0          70   
1          8         350.0       165.0  3693.0          11.5          70   
2          8         318.0       150.0  3436.0          11.0          70   
3          8         304.0       150.0  3433.0          12.0          70   
4          8         302.0       140.0  3449.0          10.5          70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64
Varianza explicada por la compontente cylinders 0.997561506349
Varianza explicada por la compontente displacement 0.00206279690861
Varianza explicada por la compontente horsepower 0.000355947852483
Varianza explicada por la compontente weight 1.49502846798e-05
Varianza explicada por la compontente acceleration 3.92231414861e-06
Varianza explicada por la compontente model_year 5.16380051433e-0

La mayor parte de la varianza viene explicada por la variable cilindros
### Procedemos a generar el modelo de regresion lineal

In [3]:
# Eliminamos caracteristicas que sean comb. lineal del resto mediante VIF
from sklearn.linear_model import LinearRegression

def calculateVIF(data):
    features = list(data.columns)
    num_features = len(features)
    
    model = LinearRegression()
    
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

def selectDataUsingVIF(data, max_VIF = 5):
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    while VIF.as_matrix().max() > max_VIF:
        col_max = np.where(VIF == VIF.as_matrix().max())[1][0]
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        VIF = calculateVIF(result)
        
    return result

calculateVIF(autos_df)

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
VIF,10.737535,21.836792,9.943693,10.83126,2.625806,1.244952,1.772386


In [4]:
model_vars = selectDataUsingVIF(autos_df)
calculateVIF(model_vars)

,cylinders,acceleration,model_year,origin
VIF,1.999959,1.384478,1.159429,1.495041


In [5]:
# Dividimos los datos en conjunto de test y entrenamiento
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(model_vars, autos_df_y)

print(X_train.describe())
print(X_test.describe())
print(y_train.describe())
print(y_test.describe())

        cylinders  acceleration  model_year      origin
count  294.000000    294.000000  294.000000  294.000000
mean     5.496599     15.428571   75.993197    1.547619
std      1.728596      2.814565    3.729774    0.789940
min      3.000000      8.000000   70.000000    1.000000
25%      4.000000     13.500000   73.000000    1.000000
50%      4.000000     15.300000   76.000000    1.000000
75%      8.000000     17.000000   79.000000    2.000000
max      8.000000     24.800000   82.000000    3.000000
       cylinders  acceleration  model_year     origin
count  98.000000     98.000000   98.000000  98.000000
mean    5.397959     15.879592   75.938776   1.663265
std     1.641848      2.568644    3.560425   0.848818
min     3.000000      8.500000   70.000000   1.000000
25%     4.000000     14.200000   73.000000   1.000000
50%     4.000000     15.750000   75.500000   1.000000
75%     6.000000     17.575000   79.750000   2.000000
max     8.000000     23.700000   82.000000   3.000000
count    2

In [6]:
# Generamos el modelo y lo evaluamos
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

modeloLineal = LinearRegression()
modeloLineal.fit(X_train, y_train)
prediccion = modeloLineal.predict(X_train)

print('R^2', modeloLineal.score(X_train, y_train))
print('Error cuadrático medio', mean_squared_error(prediccion, y_train))
print('Error absoluto medio', mean_absolute_error(prediccion, y_train))
print('Mediana del error absoluto', median_absolute_error(prediccion, y_train))
print('-----------------------------------------------------------------------------')
prediccion_test = modeloLineal.predict(X_test)
for index in range(len(prediccion_test)):
    print('prediccion:', prediccion_test[index], '; Real:', y_test.iloc[index])


R^2 0.748901684483
Error cuadrático medio 15.3930209539
Error absoluto medio 2.99726674241
Mediana del error absoluto 2.38896121829
-----------------------------------------------------------------------------
prediccion: 32.674266932 ; Real: 33.8
prediccion: 33.7608813464 ; Real: 32.9
prediccion: 20.9918796454 ; Real: 22.5
prediccion: 35.836406723 ; Real: 23.7
prediccion: 24.5852747619 ; Real: 19.0
prediccion: 17.3846805124 ; Real: 19.0
prediccion: 24.9956825811 ; Real: 28.0
prediccion: 22.0892341003 ; Real: 26.0
prediccion: 32.7233495379 ; Real: 31.3
prediccion: 27.7813133715 ; Real: 23.8
prediccion: 13.0108692083 ; Real: 14.0
prediccion: 17.581010936 ; Real: 19.0
prediccion: 23.0103724495 ; Real: 26.0
prediccion: 26.689306667 ; Real: 26.0
prediccion: 19.5058072991 ; Real: 16.0
prediccion: 17.5319283301 ; Real: 18.0
prediccion: 23.30346329 ; Real: 22.0
prediccion: 26.5911414552 ; Real: 24.0
prediccion: 13.7342723391 ; Real: 14.0
prediccion: 19.6039725108 ; Real: 19.0
prediccion: 29.0

Observamos que el R2 es bajo y el ECM muy grande. 
Ejecutamos el modelo sin quitar ninguna caracteristica para observar sus resultados.

In [10]:
autos_df_ori_y = autos_df_y

autos_df_ori_train, autos_df_ori_test,autos_df_ori_train_y, autos_df_ori_test_y = train_test_split(autos_df_ori, autos_df_ori_y)

modeloLineal.fit(autos_df_ori_train, autos_df_ori_train_y)
prediccion_ori = modeloLineal.predict(autos_df_ori_train)

print('R^2', modeloLineal.score(autos_df_ori_train, autos_df_ori_train_y))
print('Error cuadrático medio', mean_squared_error(prediccion_ori, autos_df_ori_train_y))
print('Error absoluto medio', mean_absolute_error(prediccion_ori, autos_df_ori_train_y))
print('Mediana del error absoluto', median_absolute_error(prediccion_ori, autos_df_ori_train_y))
print('-----------------------------------------------------------------------------')
prediccion_ori_test = modeloLineal.predict(autos_df_ori_test)
for index in range(len(prediccion_ori_test)):
    print('prediccion:', prediccion_ori_test[index], '; Real:', autos_df_ori_test_y.iloc[index])

R^2 0.80988428337
Error cuadrático medio 11.5401868502
Error absoluto medio 2.58312692747
Mediana del error absoluto 2.13344937447
-----------------------------------------------------------------------------
prediccion: 15.8389353967 ; Real: 16.0
prediccion: 29.493970601 ; Real: 28.0
prediccion: 31.2214643571 ; Real: 37.0
prediccion: 22.1093377389 ; Real: 20.6
prediccion: 26.4154546631 ; Real: 26.4
prediccion: 31.8615465785 ; Real: 36.0
prediccion: 23.9978291862 ; Real: 25.0
prediccion: 24.8623944597 ; Real: 19.0
prediccion: 25.0744925127 ; Real: 26.5
prediccion: 20.6927954766 ; Real: 17.5
prediccion: 25.3727686266 ; Real: 25.0
prediccion: 26.8719599633 ; Real: 26.0
prediccion: 21.0200138937 ; Real: 21.0
prediccion: 17.0734991895 ; Real: 15.5
prediccion: 16.0621912355 ; Real: 15.5
prediccion: 28.3031201232 ; Real: 24.2
prediccion: 34.2774658652 ; Real: 36.0
prediccion: 28.7865147703 ; Real: 25.4
prediccion: 17.0803194061 ; Real: 13.0
prediccion: 35.7097502633 ; Real: 37.0
prediccion: 

Se observa gran mejoria en el R2 y en ECM.